In [1]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler

## Ejercicio 2: Análisis de Funciones de Activación y Loss en Predicción de Series Temporales

Utilizarás el dataset público de calidad del aire de Beijing para crear un modelo de predicción de niveles de contaminación. Analizarás el impacto de diferentes funciones de activación y pérdida:

1. Preprocesamiento de datos:
   - Carga el dataset de calidad del aire de Beijing
   - Maneja valores faltantes
   - Normaliza las características
   - Crea secuencias temporales con ventana deslizante

In [4]:
# Lista de archivos CSV en la carpeta (ajusta el path si es necesario)
# archivos = glob.glob("C:/Users/albaro/Documents/DataSets/beijingPollution/cities/*.csv")  # Puedes cambiar esto por una lista de nombres de archivos
archivos = glob.glob("/home/albaro/Documentos/data/beijingPollution/ciudades/*.csv")
# Cargar y concatenar todos los datasets
df = pd.concat([pd.read_csv(archivo) for archivo in archivos], ignore_index=True)

# Guardar el dataset combinado
df.to_csv("dataset_completo.csv", index=False)


In [5]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
1,2,2013,3,1,1,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,NW,9.4,Shunyi
2,3,2013,3,1,2,14.0,14.0,NaN,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,NW,8.6,Shunyi
3,4,2013,3,1,3,12.0,12.0,3.0,5.0,NaN,NaN,-2.1,1027.3,-23.3,0.0,NW,6.6,Shunyi
4,5,2013,3,1,4,12.0,12.0,3.0,NaN,200.0,11.0,-2.4,1027.7,-22.9,0.0,NW,4.5,Shunyi


In [6]:
# -------------------------------- QUITAR NULOS --------------------------------

# manejar los nulos con la mediana
# df.fillna(df.median(numeric_only=True), inplace=True)

# ffill para coger los anterires datos y rellenarlos por los null
df.ffill(inplace=True)

# Comprobar nulos
print(df.isnull().sum())

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64


In [7]:
# -------------------------------- NORMALIZAR --------------------------------
# Definir columnas a normalizar (todas menos "wd", "station" y "year")
columnas_a_normalizar = df.drop(columns=["No","wd", "station", "year","month","day"]).columns

# Aplicar Min-Max Scaling
scaler = MinMaxScaler()
df[columnas_a_normalizar] = scaler.fit_transform(df[columnas_a_normalizar])

In [8]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0.000000,0.001003,0.004012,0.005432,0.024132,0.020202,0.040891,0.308943,0.718543,0.315862,0.0,NW,0.704545,Shunyi
1,2,2013,3,1,0.043478,0.010030,0.010030,0.005432,0.020671,0.020202,0.043693,0.305691,0.723510,0.304828,0.0,NW,0.712121,Shunyi
2,3,2013,3,1,0.086957,0.012036,0.012036,0.005432,0.020671,0.010101,0.020346,0.295935,0.725166,0.281379,0.0,NW,0.651515,Shunyi
3,4,2013,3,1,0.130435,0.010030,0.010030,0.005432,0.013750,0.010101,0.020346,0.289431,0.743377,0.277241,0.0,NW,0.500000,Shunyi
4,5,2013,3,1,0.173913,0.010030,0.010030,0.005432,0.013750,0.010101,0.010073,0.284553,0.750000,0.282759,0.0,NW,0.340909,Shunyi


In [9]:
# -------------------------------- SECUENCIA TEMPORAL --------------------------------

# Supongamos que df es tu DataFrame original

# Definir las columnas a utilizar (puedes incluir 'year', 'month', 'day', 'hour' si deseas)
columnas_a_utilizar = ["year", "month", "day", "hour", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]

# Crear las características (X) y las etiquetas (y)
X = df[columnas_a_utilizar].iloc[:-1]  # Usamos todos los datos menos el último (X)
y = df["PM2.5"].iloc[1:]  # Usamos los datos de PM2.5 desplazados un paso para predecir el siguiente valor

# Ver las primeras filas para comprobar que las características y etiquetas se han creado correctamente
print(X.head())
print(y.head())

   year  month  day      hour      PM10       SO2       NO2        CO  \
0  2013      3    1  0.000000  0.004012  0.005432  0.024132  0.020202   
1  2013      3    1  0.043478  0.010030  0.005432  0.020671  0.020202   
2  2013      3    1  0.086957  0.012036  0.005432  0.020671  0.010101   
3  2013      3    1  0.130435  0.010030  0.005432  0.013750  0.010101   
4  2013      3    1  0.173913  0.010030  0.005432  0.013750  0.010101   

         O3      TEMP      PRES      DEWP  RAIN      WSPM  
0  0.040891  0.308943  0.718543  0.315862   0.0  0.704545  
1  0.043693  0.305691  0.723510  0.304828   0.0  0.712121  
2  0.020346  0.295935  0.725166  0.281379   0.0  0.651515  
3  0.020346  0.289431  0.743377  0.277241   0.0  0.500000  
4  0.010073  0.284553  0.750000  0.282759   0.0  0.340909  
1    0.010030
2    0.012036
3    0.010030
4    0.010030
5    0.009027
Name: PM2.5, dtype: float64


2. Implementa una red LSTM con:
   - 2 capas LSTM
   - Capas densas intermedias
   - Capa de salida para predicción

3. Experimenta con las siguientes combinaciones:
   Funciones de activación en capas intermedias:
   - ReLU
   - LeakyReLU
   - SELU
   - Tanh
   
   Funciones de pérdida:
   - MSE
   - MAE
   - Huber Loss
   - Log-cosh

4. Analiza y visualiza:
   - Error de predicción (RMSE y MAE)
   - Estabilidad del entrenamiento
   - Capacidad de generalización
   - Comportamiento con outliers